<a href="https://colab.research.google.com/github/nextt01/wie2/blob/main/Copy_of_wie1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd /content/yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-278-g050c72c Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/78.2 GB disk)


In [3]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

In [5]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 500  --epochs 50 --data safety.yaml --weights yolov5s.pt

2024-01-26 07:33:24.823120: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-26 07:33:24.823165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-26 07:33:24.824489: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=safety.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=500, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, opt

In [11]:
!python detect.py --weights /content/yolov5/runs/train/exp2/weights/best.pt --img 640 --conf 0.25 --source test

detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=test, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-278-g050c72c Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/6 /content/yolov5/test/b.jpg: 640x448 1 glasses, 1 hat, 48.2ms
image 2/6 /content/yolov5/test/b1.jpg: 640x640 4 hats, 11.7ms
image 3/6 /content/yolov5/test/b2.jpg: 640x640 1 glasses, 1 hat, 11.5ms
image 4/6 /content/yolov5/test/b3.jpg: 640x640 1 hat, 11.5ms
image 5/6 /content/yolov5/test/b4.jpg: 640x512 1 glasses, 46.9

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
import cv2

# โหลดโมเดลตรวจจับวัตถุ
model = cv2.dnn.readNetFromTensorflow('/content/yolov5/runs/train/exp/weights/best.pt')

# เปิดกล้องเว็บแคม
cap = cv2.VideoCapture(0)

while True:
    # อ่านเฟรมจากกล้องเว็บแคม
    ret, frame = cap.read()

    # แปลงเฟรมเป็นสี RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # ตรวจจับวัตถุในเฟรม
    boxes, labels, scores = model.detectMultiScale(rgb)

    # วาดกรอบสี่เหลี่ยมรอบวัตถุที่ตรวจพบ
    for box, label, score in zip(boxes, labels, scores):
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # แสดงเฟรม
    cv2.imshow('frame', frame)

    # รอรับคีย์กด
    k = cv2.waitKey(1)

    # กด q เพื่อออกจากโปรแกรม
    if k == ord('q'):
        break

# ปิดกล้องเว็บแคม
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.8.0) /io/opencv/modules/dnn/src/tensorflow/tf_io.cpp:42: error: (-2:Unspecified error) FAILED: ReadProtoFromBinaryFile(param_file, param). Failed to parse GraphDef file: /content/yolov5/runs/train/exp/weights/best.pt in function 'ReadTFNetParamsFromBinaryFileOrDie'
